<a href="https://colab.research.google.com/github/fadhlijahfal1/Big-Data-Analytic/blob/main/Spark_Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Catatan:**

Series = Array 1 dimensi

Dataframe = 2 dimensi (matriks)

Install Java dan Spark

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz
!pip install -q findspark

Set Environment Variabels

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"

In [3]:
!ls

sample_data		 spark-3.5.1-bin-hadoop3.tgz  Tree_Environment.csv
spark-3.5.1-bin-hadoop3  Tree.csv


In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

Contoh Penggunaan Pyspark Sederhana

In [5]:
data = [("Budi", 22), ("Apik", 81), ("Aji", 25)]

df = spark.createDataFrame(data, ["Nama", "Usia"])

df.show()

+----+----+
|Nama|Usia|
+----+----+
|Budi|  22|
|Apik|  81|
| Aji|  25|
+----+----+



Load Dataset

In [10]:
trees_df = spark.read.csv("/content/Tree.csv", header=True, inferSchema=True)

trees_df.show()

+-------+-------------------+--------+---------+-----------+--------------+------------+---------+---------------+-------------+
|Tree_ID|            Species|Height_m|Age_years|Diameter_cm|      Location|Planted_Date|Evergreen|Disease_Present|Health_Status|
+-------+-------------------+--------+---------+-----------+--------------+------------+---------+---------------+-------------+
|      1|      Quercus robur|    25.3|       45|      120.5|  North Forest|   3/15/1978|    false|           true|         Sick|
|      2|   Pinus sylvestris|    32.1|       60|       95.8|Mountain Range|   7/22/1963|     true|          false|      Healthy|
|      3|Acer pseudoplatanus|    18.7|       30|       68.2|     City Park|   11/5/1993|    false|           true|         Sick|
|      4|    Fagus sylvatica|    28.9|       55|      112.3|   South Slope|   9/12/1968|    false|          false|      Healthy|
|      5|        Picea abies|    41.5|       85|      145.0|   Alpine Zone|   5/30/1938|     true

In [13]:
environment_df = spark.read.csv("/content/Tree_Environment.csv", header = True, inferSchema = True)

environment_df.show()

+-------+---------+------------------+-----------------+--------------------------+-----------+
|Tree_ID|Soil_Type|Annual_Rainfall_mm|Avg_Temperature_C|Carbon_Sequestration_kg_yr|     Family|
+-------+---------+------------------+-----------------+--------------------------+-----------+
|      1|     Loam|              1200|              8.5|                      25.3|   Fagaceae|
|      2|    Sandy|               800|              6.2|                      38.7|   Pinaceae|
|      3|     Clay|               950|             10.1|                      15.9|Sapindaceae|
|      4|     Loam|              1100|              7.8|                      28.4|   Fagaceae|
|      5|    Rocky|               650|              4.5|                      42.1|   Pinaceae|
|      6|    Sandy|               900|              9.3|                      10.2| Betulaceae|
|      7|     Clay|              1050|              8.9|                      32.8|   Pinaceae|
|      8|     Loam|              1150|  

Basic DataFrame Operations

In [14]:
# Menampilkan 5 Baris Data
trees_df.show(5)

+-------+-------------------+--------+---------+-----------+--------------+------------+---------+---------------+-------------+
|Tree_ID|            Species|Height_m|Age_years|Diameter_cm|      Location|Planted_Date|Evergreen|Disease_Present|Health_Status|
+-------+-------------------+--------+---------+-----------+--------------+------------+---------+---------------+-------------+
|      1|      Quercus robur|    25.3|       45|      120.5|  North Forest|   3/15/1978|    false|           true|         Sick|
|      2|   Pinus sylvestris|    32.1|       60|       95.8|Mountain Range|   7/22/1963|     true|          false|      Healthy|
|      3|Acer pseudoplatanus|    18.7|       30|       68.2|     City Park|   11/5/1993|    false|           true|         Sick|
|      4|    Fagus sylvatica|    28.9|       55|      112.3|   South Slope|   9/12/1968|    false|          false|      Healthy|
|      5|        Picea abies|    41.5|       85|      145.0|   Alpine Zone|   5/30/1938|     true

In [16]:
# Menampilkan Pohon Yang TIngginya Lebih dari 30M dan Berumur Kurang dari 70 Tahun
tall_young_trees = trees_df.filter((trees_df.Height_m > 30) & (trees_df.Age_years < 70))

tall_young_trees.show()

+-------+----------------+--------+---------+-----------+--------------+------------+---------+---------------+-------------+
|Tree_ID|         Species|Height_m|Age_years|Diameter_cm|      Location|Planted_Date|Evergreen|Disease_Present|Health_Status|
+-------+----------------+--------+---------+-----------+--------------+------------+---------+---------------+-------------+
|      2|Pinus sylvestris|    32.1|       60|       95.8|Mountain Range|   7/22/1963|     true|          false|      Healthy|
|     12| Pinus ponderosa|    31.9|       65|      110.2|  North Forest|   6/14/1958|     true|          false|      Healthy|
|     19|   Abies procera|    34.9|       68|      123.8|Mountain Range|   10/5/1955|     true|          false|      Healthy|
+-------+----------------+--------+---------+-----------+--------------+------------+---------+---------------+-------------+



In [17]:
# Menampilkan Jumlah Dengan Status Kesehatan
health_counts = trees_df.groupBy("Health_Status").count()

health_counts.show()

+-------------+-----+
|Health_Status|count|
+-------------+-----+
|         NULL|    1|
|         Dead|    3|
|         Sick|    7|
|      Healthy|   10|
+-------------+-----+



Data Cleaning

In [18]:
# Cek Missing Values
from pyspark.sql.functions import col, sum

missing_values = trees_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in
trees_df.columns])

missing_values.show()

+-------+-------+--------+---------+-----------+--------+------------+---------+---------------+-------------+
|Tree_ID|Species|Height_m|Age_years|Diameter_cm|Location|Planted_Date|Evergreen|Disease_Present|Health_Status|
+-------+-------+--------+---------+-----------+--------+------------+---------+---------------+-------------+
|      0|      0|       1|        0|          0|       0|           0|        0|              0|            1|
+-------+-------+--------+---------+-----------+--------+------------+---------+---------------+-------------+



In [19]:
# Isi Missing Values di Height_m dengan Median
from pyspark.sql.functions import median

median_height = trees_df.select(median("Height_m")).first()[0]

trees_df = trees_df.na.fill(median_height, subset = ["Height_m"])

trees_df.show()

+-------+-------------------+--------+---------+-----------+--------------+------------+---------+---------------+-------------+
|Tree_ID|            Species|Height_m|Age_years|Diameter_cm|      Location|Planted_Date|Evergreen|Disease_Present|Health_Status|
+-------+-------------------+--------+---------+-----------+--------------+------------+---------+---------------+-------------+
|      1|      Quercus robur|    25.3|       45|      120.5|  North Forest|   3/15/1978|    false|           true|         Sick|
|      2|   Pinus sylvestris|    32.1|       60|       95.8|Mountain Range|   7/22/1963|     true|          false|      Healthy|
|      3|Acer pseudoplatanus|    18.7|       30|       68.2|     City Park|   11/5/1993|    false|           true|         Sick|
|      4|    Fagus sylvatica|    28.9|       55|      112.3|   South Slope|   9/12/1968|    false|          false|      Healthy|
|      5|        Picea abies|    41.5|       85|      145.0|   Alpine Zone|   5/30/1938|     true